In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend
%load_ext autoreload
%autoreload 2

InlineBackend(InlineBackendConfig) options
----------------------------------------
InlineBackend.close_figures=<Bool>
    Close all figures at the end of each cell.
            When True, ensures that each cell starts with no active figures, but it
            also means that one must keep track of references in order to edit or
            redraw figures in subsequent cells. This mode is ideal for the notebook,
            where residual plots from other cells might be surprising.
            When False, one must call figure() to create new figures. This means
            that gcf() and getfigs() can reference figures created in other cells,
            and the active figure can continue to be edited with pylab/pyplot
            methods that reference the current active figure. This mode facilitates
            iterative editing of figures, and behaves most consistently with
            other matplotlib backends, but figure barriers between cells must
            be explicit.
    Cu

In [3]:
header = ['wiki_movie_id','freebase_movie_id','movie_release_date','character_name','actor_date_of_birth','actor_gender','actor_height','actor_ethnicity','actor_name','actor_age_at_release','freebase_character_actor_map_id','freebase_character_id','freebase_actor_id']
df = pd.read_csv('dataset/character.metadata.tsv',sep='\t',names=header)
df.head()

,wiki_movie_id,freebase_movie_id,movie_release_date,character_name,actor_date_of_birth,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age_at_release,freebase_character_actor_map_id,freebase_character_id,freebase_actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [55]:
import xml.etree.ElementTree as ET
import gzip
import xmltodict

# Load and parse the XML file
directory = 'dataset/corenlp_plot_summaries/'
file_path = directory + '23270459' + '.xml.gz'

In [102]:
def extract_corenlp_xml(filename):
    tree = ET.parse(gzip.open(filename))
    #xml to json
    xmlstr = ET.tostring(tree.getroot(), encoding='utf8', method='xml')
    data_dict = xmltodict.parse(xmlstr)
    return data_dict

In [103]:
file = extract_corenlp_xml(file_path)

In [159]:
#print file structure two levels
for coreference in file['root']['document']['coreference']['coreference']:
    # find token
    for mention in coreference['mention']:
        
        sentence = file['root']['document']['sentences']['sentence'][int(mention['sentence'])-1] # sentence index in mention start from 1
        for token in sentence['tokens']['token']:
            if int(mention['start']) <= int(token['@id']) < int(mention['end']):
                print(token['word'].upper(),end=' ')
            else:   
                print(token['word'],end=' ')
        print('')
    print('\n')

FORMER DREAM ARCHITECT DOMINICK `` DOM '' COBB and business partner Arthur perform corporate espionage by using an experimental military-developed machine to infiltrate the subconscious of their targets and extract information while dreaming , their latest target being powerful Japanese businessman Saito . 
Former dream architect Dominick `` Dom '' COBB and business partner Arthur perform corporate espionage by using an experimental military-developed machine to infiltrate the subconscious of their targets and extract information while dreaming , their latest target being powerful Japanese businessman Saito . 
COBB 'S totem is a spinning top that perpetually spins in the dream state . 
The extraction fails due to MALLORIE `` MAL '' COBB , COBB 'S DECEASED WIFE , WHOSE MEMORY PROJECTION SABOTAGES THE MISSION . 
The extraction fails due to MALLORIE `` MAL '' COBB , Cobb 's deceased wife , whose memory projection sabotages the mission . 
The extraction fails due to Mallorie `` Mal '' COBB

In [109]:
def link_token_to_character(token, entities, coreferences):
    # Assume `coreferences` is a dictionary mapping token indices to character names
    token_index = token['id']  # The unique identifier for the token

    # Check if the token index is in the coreference mapping
    if token_index in coreferences:
        # If so, return the associated character name
        return coreferences[token_index]

    # If the token is not in the coreference map, check its text directly against entity names
    for entity in entities:
        if token['word'] == entity['text']:
            return entity['text']

    # If no character is found, return None
    return None


def extract_character_complexity(data_dict):
    characters = {}  # Dictionary to hold character information

    # Loop through sentences to extract entities and their actions
    for sentence in data_dict['root']['document']['sentences']['sentence']:
        entities = sentence.get('entities', {}).get('entity', [])
        tokens = sentence['tokens']['token']

        # Extract entities and classify them as characters
        for entity in entities:
            if entity['type'] == 'PERSON':  # Assuming 'type' is the correct key for entity type
                char_name = entity['text']
                if char_name not in characters:
                    characters[char_name] = {
                        'actions': [],
                        'descriptions': [],
                        'dialogue': []
                    }

        # Extract actions and descriptions associated with characters
        for token in tokens:
            # Assuming there's a way to link tokens to the character entities, possibly through coreference chains
            # Pseudocode for the linking part
            char_name = link_token_to_character(token, entities, data_dict['coreferences'])
            if char_name:
                if token['POS'] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:  # Verbs
                    characters[char_name]['actions'].append(token['word'])
                elif token['POS'] in ['JJ', 'JJR', 'JJS']:  # Adjectives
                    characters[char_name]['descriptions'].append(token['word'])
                # Add more conditions as needed for dialogue, etc.

    return characters


In [110]:
extract_character_complexity(file)

KeyError: 'coreferences'